# script COTO

In [40]:
import pandas as pd

In [41]:
articulos = {'coto':
                {'salamin': 'https://www.cotodigital3.com.ar/sitios/cdigi/producto/-salamin-tandfin-cagnoli-xkg/_/A-00080962-00080962-200', 
                'cerveza': 'https://www.cotodigital3.com.ar/sitios/cdigi/producto/-cerveza-fresquita-andes-origen-473-ml/_/A-00545197-00545197-200', 
                'papa_fritas': 'https://www.cotodigital3.com.ar/sitios/cdigi/producto/-papas-fritas-clasicas-lays-40g/_/A-00567578-00567578-200'
                },
            'jumbo':
                {'cerveza': 'https://www.jumbo.com.ar/cerveza-andes-origen-roja-botella-retornable-1ltx1/p'
                
                },                  
            'carrefour': 
                {'cerveza': 'https://www.carrefour.com.ar/cerveza-rubia-andes-origen-473-cc/p'}
           }

In [42]:
# def get_coto(url):
    
#     headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36'}    
#     response = requests.get(url, headers=headers)
    
#     soup = BeautifulSoup(response.content, 'html.parser')
    
#     product_name = soup.find('h1', class_='product_page').get_text(strip=True)
#     product_price = soup.find('span', class_='atg_store_newPrice').get_text(strip=True).replace('$', '').replace('.', '').replace(',', '.')

#     print(f'Product Name: {product_name}')
#     print(f'Price: {product_price}')
#     return product_price

def get_coto(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36'
    }
    
    try:
        response = requests.get(url, headers=headers)        
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            
            product_name = soup.find('h1', class_='product_page').get_text(strip=True)
            product_price = soup.find('span', class_='atg_store_newPrice').get_text(strip=True).replace('$', '').replace('.', '').replace(',', '.')
            
            print(f'Product Name: {product_name}')
            print(f'Price: {product_price}')
            return product_price
        
        else:
            print(f'Failed to retrieve webpage, status code: {response.status_code}')
            return None
    
    except requests.RequestException as e:
        print(f'Request failed: {e}')
        return None
    
    
def get_jumbo(url):    
    
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36'}
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        script = soup.find('script', {'type': 'application/ld+json'})
        
        if script:
            data = json.loads(script.string)
            product_name = data.get('name')
            product_price = data.get('offers', {}).get('lowPrice')

            print(f'Product Name: {product_name}')
            print(f'Price: {product_price}')
            return product_price
        
        else:
            print('Product data in JSON-LD format not found.')
    else:
        print(f'Failed to retrieve webpage, status code: {response.status_code}')
        
        
def create_df(articulos, product_key):
    data_dict = {}
    
    store_function_mapping = {
        'coto': get_coto,
        'jumbo': get_jumbo,
        'carrefour': get_jumbo
    }
    
    for store, products in articulos.items():
        if product_key in products:
            
            if store in store_function_mapping:  
                get_function = store_function_mapping[store]
                product_price = get_function(products[product_key])
                data_dict[store] = product_price
                
    df = pd.DataFrame([data_dict])
    
    return df

In [43]:
df_cerveza_prices = create_df(articulos, 'cerveza')
df_cerveza_prices

Product Name: Cerveza Fresquita Andes Origen 473 Ml
Price: 1499.00
Product Name: Cerveza Andes Origen Roja Botella Retornable 1ltx1
Price: 2712
Product Name: Cerveza rubia Andes Origen 473 cc.
Price: 1119.3


,coto,jumbo,carrefour
0,1499.00,2712,1119.3
